# extracting accessibility related patterns from ravelry

%%
## bootstrap our search for accessible ravelry patterns

there are several accessibility categories that need to be searched.
they are captured in the `searches` mapping where the key is the name/index
for the search and the value defines the query parameters.

In [5]:
with __import__("importnb").Notebook():
    from app import App
import os, time, random, operator
from toolz.curried import compose, do, partial
from pandas import Series

In [6]:
def get_search_cache(app):
    return (app.cache / "ravelry" / "search").client()
def get_pattern_cache(app):
    return (app.cache / "ravelry" / "patterns").client()

In [7]:
searches: dict = {
    "adaptive": {"pa": "adaptive"},
    "medical device access": {"pa": "medical-device-access"},
    "medical device support": {"pa": "medical-device-accessory"},
    # "mobility aid support": {"add": "mobility-aid-support"},
    # "other": {"add": "other-add-accessibility"},
    "therapy aid/toy": {"pa": "therapy-aid"},
    "medical": {"pc": "medical"}
}
@App.impl
def get_responses(app):
    auth = (os.environ["RAVELRY_USERNAME"], os.environ["RAVELRY_PASSWORD"])
    search_cache = get_search_cache(app)
    
    seed_urls = ("https://api.ravelry.com/patterns/search.json?" + Series(searches).apply(urllib.parse.urlencode))
    first_pages = seed_urls.apply(compose(do(print), search_cache.get), auth=auth)
    first_patterns = first_pages.apply(operator.methodcaller("json")).apply(Series)
    paginated = first_patterns.paginator.apply(Series)
    paginated = paginated[paginated.page_count.gt(1)].drop(columns="page")
    paginated = paginated.join(paginated.page_count.add(1).apply(compose(list, partial(range, 2))).explode().rename("page"))
    other_urls = seed_urls[paginated.index] + "&page=" + paginated.page.astype(str)
    other_urls.apply(compose(do(print), search_cache.get), auth=auth)
    get_pattern_requests(app, seed_urls, other_urls)

def get_pattern_requests(app, *df):
    search_cache = get_search_cache(app)
    patterns = (
        pandas.concat(df)
        .apply(search_cache.get)
        .apply(operator.methodcaller("json"))
        .apply(operator.itemgetter("patterns"))
        .explode()
        .apply(Series)
        .drop_duplicates("permalink")
    )
    pattern_urls = patterns.set_index(("https://www.ravelry.com/patterns/library/" + patterns["permalink"]).rename("url"))
    patterns = get_responses(app)
    pattern_chuck_size = 20
    pattern_urls = "https://api.ravelry.com/patterns.json?ids=" + (
        patterns.id.groupby(pandas.RangeIndex(len(patterns)) // pattern_chuck_size)
        .agg(list)
        .apply(compose("+".join, map(str)))
    )
    patterns_cache = get_pattern_cache(app)
    (
        pattern_urls.apply(compose(patterns_cache.get), auth=auth)
        .apply(operator.methodcaller("json"))
        .apply(operator.itemgetter("patterns"))
        .apply(compose(list, dict.values))
        .explode()
        .apply(Series)
    )
    
@App.impl
def get_frames(app):
    return (
        df := Series(get_search_cache(app).cache.responses)
        .apply(operator.methodcaller("json"))
        .apply(operator.itemgetter("patterns"))
        .explode()
        .apply(Series)
        .set_index("id")
    ).join(
        (
            g:= Series(get_pattern_cache(app).cache.responses)
            .apply(operator.methodcaller("json"))
            .apply(operator.itemgetter("patterns"))
            .apply(dict.values)
            .explode()
            .apply(Series)
            .set_index("id")
        )[g.columns.difference(df.columns)]
    )

In [8]:
if __name__ == "__main__":
    app = App()
    app.main(__import__(__name__), run=0 or locals().get("__file__") is not None)
    df = app.compact()
    display(
        df.T,
    )
        

id,1059854,1071789,1091183,1112262,1112336,1112733,1119218,1130885,1183021,1208154,...,982660,984003,984286,986552,986825,988405,992731,1004598,1006146,1007543
free,True,False,True,True,True,True,False,True,False,True,...,True,False,False,True,True,True,True,True,False,False
name,Groovy Face Mask Lanyards,Knitt Ear Saver,Completely Hooked Lanyard,Stroke driver bags,Adaptive Adult Thumbless Mittens,Bulky Adaptive Thumbless Mittens,Variety Ear Savers for Face Masks,Mio's Mask Extender Ear Savers,Too Hot To Handle Bag,Chemo Strap & Holder,...,Knit Packer,Monsieur Louis,JT Necktie,Quick & Easy Round Knocker,Unplanned Extubation Prevention Mitts,Senior socks with handles,Realistic soft packer/ willy,Chill Buddy Ice Pack,I Love You in Braille,I Love You in Braille Knit Washcloth
permalink,groovy-face-mask-lanyards,knitt-ear-saver,completely-hooked-lanyard,stroke-driver-bags,adaptive-adult-thumbless-mittens,bulky-adaptive-thumbless-mittens,variety-ear-savers-for-face-masks,mios-mask-extender-ear-savers,too-hot-to-handle-bag,chemo-strap--holder,...,knit-packer,monsieur-louis,jt-necktie,quick--easy-round-knocker,unplanned-extubation-prevention-mitts,senior-socks-with-handles,realistic-soft-packer--willy,chill-buddy-ice-pack,i-love-you-in-braille,i-love-you-in-braille-knit-washcloth
personal_attributes,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
first_photo,"{'id': 96615983, 'sort_order': 1, 'user_id': 4...","{'id': 97722032, 'sort_order': 1, 'user_id': 8...","{'id': 99416899, 'sort_order': 1, 'user_id': 1...","{'id': 101641353, 'sort_order': 1, 'user_id': ...","{'id': 101647374, 'sort_order': 1, 'user_id': ...","{'id': 101684802, 'sort_order': 1, 'user_id': ...","{'id': 102277277, 'sort_order': 1, 'user_id': ...","{'id': 103365201, 'sort_order': 1, 'user_id': ...","{'id': 107623644, 'sort_order': 1, 'user_id': ...","{'id': 109859573, 'sort_order': 1, 'user_id': ...",...,"{'id': 89220179, 'sort_order': 1, 'user_id': 1...","{'id': 89329529, 'sort_order': 1, 'user_id': 3...","{'id': 89363003, 'sort_order': 1, 'user_id': 4...","{'id': 89580464, 'sort_order': 1, 'user_id': 3...","{'id': 89607199, 'sort_order': 1, 'user_id': 2...","{'id': 89754909, 'sort_order': 1, 'user_id': 6...","{'id': 90355998, 'sort_order': 1, 'user_id': 9...","{'id': 91502062, 'sort_order': 1, 'user_id': 2...","{'id': 91654487, 'sort_order': 1, 'user_id': 1...","{'id': 91791547, 'sort_order': 1, 'user_id': 1..."
designer,"{'crochet_pattern_count': 5, 'favorites_count'...","{'crochet_pattern_count': 5, 'favorites_count'...","{'crochet_pattern_count': 163, 'favorites_coun...","{'crochet_pattern_count': 1, 'favorites_count'...","{'crochet_pattern_count': 2, 'favorites_count'...","{'crochet_pattern_count': 2, 'favorites_count'...","{'crochet_pattern_count': 4, 'favorites_count'...","{'crochet_pattern_count': 0, 'favorites_count'...","{'crochet_pattern_count': 7, 'favorites_count'...","{'crochet_pattern_count': 1, 'favorites_count'...",...,"{'crochet_pattern_count': 0, 'favorites_count'...","{'crochet_pattern_count': 0, 'favorites_count'...","{'crochet_pattern_count': 7, 'favorites_count'...","{'crochet_pattern_count': 2, 'favorites_count'...","{'crochet_pattern_count': 0, 'favorites_count'...","{'crochet_pattern_count': 0, 'favorites_count'...","{'crochet_pattern_count': 127, 'favorites_coun...","{'crochet_pattern_count': 1013, 'favorites_cou...","{'crochet_pattern_count': 1, 'favorites_count'...","{'crochet_pattern_count': 1, 'favorites_count'..."
pattern_author,"{'crochet_pattern_count': 5, 'favorites_count'...","{'crochet_pattern_count': 5, 'favorites_count'...","{'crochet_pattern_count': 163, 'favorites_coun...","{'crochet_pattern_count': 1, 'favorites_count'...","{'crochet_pattern_count': 2, 'favorites_count'...","{'crochet_pattern_count': 2, 'favorites_count'...","{'crochet_pattern_count': 4, 'favorites_count'...","{'crochet_pattern_count': 0, 'favorites_count'...","{'crochet_pattern_count': 7, 'favorites_c